In [2]:
from llama_index.core import PromptTemplate
import json
import torch
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from pyvi import ViTokenizer
import weaviate
from weaviate.classes.init import Auth

/home/drissdo/Desktop/VN_LAW_QA_SYSTEM/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

DATA_COLLECTION = "ND168"
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "dangvantuan/vietnamese-document-embedding"
CHUNK_SIZE = 512  # Optimized for Vietnamese text
CHUNK_OVERLAP = 50  # Small overlap to maintain context

In [4]:
# Setup vector store
# client = weaviate.connect_to_weaviate_cloud(
#     cluster_url=WEAVIATE_URL,
#     auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
# )


client = weaviate.connect_to_local(
                     host="192.168.100.125",
                        port=8080,
                        grpc_port=50051
                )
vector_store = WeaviateVectorStore(
    weaviate_client=client,
    index_name=DATA_COLLECTION
)

In [5]:
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    embed_model = HuggingFaceEmbedding(model_name=MODEL_NAME, device=DEVICE, trust_remote_code=True,cache_folder="/home/drissdo/.cache/huggingface/hub"),
)

In [6]:
# Create retriever
retriever = index.as_retriever(
    vector_store_query_mode="hybrid",
    similarity_top_k=20,
    alpha=0.3,
)

In [7]:
question = "Phân biệt mức xử phạt giữa hành vi chở trẻ em dưới 10 tuổi ngồi cùng hàng ghế với người lái xe và điều khiển xe ô tô kinh doanh vận tải chở trẻ em mầm non không có dây đai an toàn?"
retrieved_docs = retriever.retrieve(question)

for i, node in enumerate(retrieved_docs):
    print(f"Result {i+1}:\n")
    print(f"Text: {node.text}\n")
    print(f"Metadata: {node.metadata}\n")
    print(f"Score: {node.score}\n")
    print("="*50)

Result 1:

Text: loại phương_tiện : ô_tô mức phạt : phạt tiền từ 6.000.000 đồng đến 8.000.000 đồng_nội_dung vi_phạm : ( đối_với tổ_chức kinh_doanh vận_tải , dịch_vụ hỗ_trợ vận_tải ) sử_dụng xe ô_tô kinh_doanh vận_tải chở trẻ_em mầm_non , học_sinh tiểu_học , xe ô_tô kinh_doanh vận_tải kết_hợp với hoạt_động đưa_đón trẻ_em mầm_non , học_sinh tiểu_học không có dây đai an_toàn phù_hợp với lứa tuổi hoặc không có ghế_ngồi phù_hợp với lứa tuổi .

Metadata: {'category': 'ô tô', 'fine_amount': 'Phạt tiền từ 6.000.000 đồng đến 8.000.000 đồng', 'violation_type': 'khác', 'original_text': 'Loại phương tiện: ô tô\nMức phạt: Phạt tiền từ 6.000.000 đồng đến 8.000.000 đồng\nNội dung vi phạm: (đối với tổ chức kinh doanh vận tải, dịch vụ hỗ trợ vận tải)Sử dụng xe ô tô kinh doanh vận tải chở trẻ em mầm non, học sinh tiểu học, xe ô tô kinh doanh vận tải kết hợp với hoạt động đưa đón trẻ em mầm non, học sinh tiểu học không có dây đai an toàn phù hợp với lứa tuổi hoặc không có ghế ngồi phù hợp với lứa tuổi.'}